In [0]:
from pyspark.sql.functions import from_unixtime, date_format, col, concat, lit
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, ArrayType

In [0]:
# Lista todos os arquivos ou diretórios no diretório bronze
arquivos = dbutils.fs.ls("/Volumes/projeto/default/bronze/")

# Filtra diretórios que começam com "clima_" e terminam com ".parquet" (mesmo com a barra no final)
clima_arquivos = [f for f in arquivos if f.name.startswith("clima_") and "parquet" in f.name]

# Ordena por data de modificação
clima_arquivos_ordenados = sorted(clima_arquivos, key=lambda f: f.modificationTime, reverse=True)

# Pega o mais recente
if clima_arquivos_ordenados:
    caminho_bronze = clima_arquivos_ordenados[0].path
    print(f"Arquivo mais recente encontrado: {caminho_bronze}")

    # Continuar com a leitura e transformação
    df_bronze = spark.read.parquet(caminho_bronze)

    df_prata = df_bronze.select(
        col("main.temp").alias("temperature"),
        col("main.feels_like").alias("real_temperature"),
        col("main.temp_min").alias("temp_min"),
        col("main.temp_max").alias("temp_max"),
        col("main.humidity").alias("humidity"),
        concat(date_format(from_unixtime(col("dt")), "yyyy-MM-dd'T'HH:mm:ss.SSSSSS"), lit("Z")).alias("timestamp"),
        col("weather").getItem(0).getField("description").alias("weather_description")
    )

    caminho_prata = "/Volumes/projeto/default/prata/climasp.parquet"

    df_prata.write.mode("overwrite").parquet(caminho_prata)
    df_prata.write.mode("overwrite").saveAsTable("clima_tabela")

    df_prata.printSchema()
    df_prata.show()
else:
    print("Nenhum arquivo válido encontrado na camada bronze.")
